In [1]:
%reset -f

In [2]:
#.\.venv\Scripts\Activate.ps1

import pandas as pd
import numpy as np
from pathlib import Path

bases_path = Path("bases")

checklist = bases_path / "HERING_FRANQUIA_checklist_RA_SMOOTH_0_2_2025-12-03.xlsx"
relatorio = bases_path / "relatorio.xlsx"
estoque = bases_path / "STK_3.12.xlsx"
carteira = bases_path / "CA_APS_3.12.xlsx"
carteira_fat = bases_path / "CA_F_3.12.xlsx"
df_check = pd.read_excel(checklist, usecols=['FILIAL', 'ARTIGO_COR', 'PRESENTE', 'TRANSITO', 'EMBALADO', 'RESERVADO'], engine="calamine")
df_relatorio = pd.read_excel(relatorio, header=5, usecols=['Cod. Loja', 'Artigo', 'Cor', 'Atendido (SAP)', 'Cancelada'], engine="calamine")
df_estoque = pd.read_excel(estoque, engine="calamine")
df_cat = pd.read_excel(carteira, engine="calamine")
df_cat_fat_hoje_ontem = pd.read_excel(carteira_fat, engine="calamine")

In [3]:
df_check.columns

Index(['FILIAL', 'ARTIGO_COR', 'PRESENTE', 'TRANSITO', 'EMBALADO',
       'RESERVADO'],
      dtype='object')

In [4]:
df_check.dtypes

FILIAL         int64
ARTIGO_COR    object
PRESENTE       int64
TRANSITO       int64
EMBALADO       int64
RESERVADO      int64
dtype: object

In [5]:
df_check['ARTIGO_COR'] = df_check['ARTIGO_COR'].str.replace('-', '')
df_check['FILIAL'] = df_check['FILIAL'].astype(str)
df_check['chave'] =  df_check['FILIAL'] + df_check['ARTIGO_COR'] 

In [6]:
df_check.head()

,FILIAL,ARTIGO_COR,PRESENTE,TRANSITO,EMBALADO,RESERVADO,chave
0,282,0201N1007S,124,26,0,0,2820201N1007S
1,978,0201N1007S,30,0,87,87,9780201N1007S
2,282,0201N1007S,109,25,10,10,2820201N1007S
3,978,0201N1007S,44,0,59,59,9780201N1007S
4,110,0227N1007S,126,0,11,11,1100227N1007S


In [7]:
df_estoque.head()

,Dia,Canal,Ponto Venda Cód,Artigo,Pirâmide Atual,Peças,Estoque Total,Estoque em Trânsito
0,2,Franquias,3,0B5DN10EN,Essencial,0,17,13
1,2,Franquias,3,0B5DNM2EN,Sazonal,0,6,0
2,2,Franquias,3,0B5DRX8EN,Sazonal,0,3,0
3,2,Franquias,3,02QYN0A00S,Sazonal,0,15,0
4,2,Franquias,3,02QYN1007S,Sazonal,0,6,0


In [8]:
df_estoque.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531589 entries, 0 to 531588
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   Dia                   531589 non-null  int64 
 1   Canal                 531589 non-null  object
 2   Ponto Venda Cód       531589 non-null  int64 
 3   Artigo                531589 non-null  object
 4   Pirâmide Atual        531589 non-null  object
 5   Peças                 531589 non-null  int64 
 6   Estoque Total         531589 non-null  int64 
 7   Estoque em Trânsito   531589 non-null  int64 
dtypes: int64(5), object(3)
memory usage: 32.4+ MB


In [9]:
df_estoque.columns

Index(['Dia ', 'Canal ', 'Ponto Venda Cód ', 'Artigo ', 'Pirâmide Atual ',
       'Peças ', 'Estoque Total ', 'Estoque em Trânsito '],
      dtype='object')

In [10]:
df_estoque.columns = df_estoque.columns.str.strip()
colunas_numericas = ['Peças', 'Estoque Total', 'Estoque em Trânsito']
colunas_categoricas = ['Canal', 'Ponto Venda Cód', 'Pirâmide Atual']
df_estoque[colunas_numericas] = df_estoque[colunas_numericas].astype('Int64')
df_estoque[colunas_categoricas] = df_estoque[colunas_categoricas].astype('str')
print("\n\n---- Depois da limpeza ---")
df_estoque.dtypes



---- Depois da limpeza ---


Dia                     int64
Canal                  object
Ponto Venda Cód        object
Artigo                 object
Pirâmide Atual         object
Peças                   Int64
Estoque Total           Int64
Estoque em Trânsito     Int64
dtype: object

In [11]:
df_estoque.columns

Index(['Dia', 'Canal', 'Ponto Venda Cód', 'Artigo', 'Pirâmide Atual', 'Peças',
       'Estoque Total', 'Estoque em Trânsito'],
      dtype='object')

In [12]:
df_estoque['chave'] =  df_estoque['Ponto Venda Cód'] + df_estoque['Artigo'] 

In [13]:
df_estoque.head()

,Dia,Canal,Ponto Venda Cód,Artigo,Pirâmide Atual,Peças,Estoque Total,Estoque em Trânsito,chave
0,2,Franquias,3,0B5DN10EN,Essencial,0,17,13,30B5DN10EN
1,2,Franquias,3,0B5DNM2EN,Sazonal,0,6,0,30B5DNM2EN
2,2,Franquias,3,0B5DRX8EN,Sazonal,0,3,0,30B5DRX8EN
3,2,Franquias,3,02QYN0A00S,Sazonal,0,15,0,302QYN0A00S
4,2,Franquias,3,02QYN1007S,Sazonal,0,6,0,302QYN1007S


In [14]:
df_estoque['STK_REAL'] = df_estoque['Estoque Total'] - df_estoque['Peças']

In [15]:
df_estoque.head()

,Dia,Canal,Ponto Venda Cód,Artigo,Pirâmide Atual,Peças,Estoque Total,Estoque em Trânsito,chave,STK_REAL
0,2,Franquias,3,0B5DN10EN,Essencial,0,17,13,30B5DN10EN,17
1,2,Franquias,3,0B5DNM2EN,Sazonal,0,6,0,30B5DNM2EN,6
2,2,Franquias,3,0B5DRX8EN,Sazonal,0,3,0,30B5DRX8EN,3
3,2,Franquias,3,02QYN0A00S,Sazonal,0,15,0,302QYN0A00S,15
4,2,Franquias,3,02QYN1007S,Sazonal,0,6,0,302QYN1007S,6


In [16]:
df_estoque_agrupado = df_estoque.groupby('chave')['STK_REAL'].sum().reset_index()
df_estoque_agrupado
total = df_estoque_agrupado['STK_REAL'].sum()

print("--- Estoque Agrupado por Chave ---")
print(df_estoque_agrupado)
print(f"\n O total de estoque real no BI é: {total}")

--- Estoque Agrupado por Chave ---
                chave  STK_REAL
0        1000111AX7EN         7
1       1000111M2H07S         3
2       1000111MD307S         3
3       1000111N0A00S        20
4       1000111N1007S        43
...               ...       ...
531584     9N3A7RY3EN        23
531585    9N3A7WA507S         1
531586     9N3A7WHBEN        15
531587     9N3A7WKAEN         1
531588     9N3A7YX7EN         3

[531589 rows x 2 columns]

 O total de estoque real no BI é: 5852667


In [17]:
TD_STK =  df_check.groupby(['chave','FILIAL', 'ARTIGO_COR'])['PRESENTE'].sum().reset_index()
TD_STK = TD_STK.merge(df_estoque[['chave', 'STK_REAL']], how='left', on='chave' )
TD_STK['STK_REAL'] = TD_STK['STK_REAL'].fillna(0).astype(np.int64)
TD_STK['DIF_ABS'] = (TD_STK['PRESENTE'] - TD_STK['STK_REAL']).abs()
TD_STK

,chave,FILIAL,ARTIGO_COR,PRESENTE,STK_REAL,DIF_ABS
0,1000111AX7EN,100,0111AX7EN,8,7,1
1,1000111M2H07S,100,0111M2H07S,3,3,0
2,1000111MD307S,100,0111MD307S,3,3,0
3,1000111N0A00S,100,0111N0A00S,21,20,1
4,1000111N1007S,100,0111N1007S,43,43,0
...,...,...,...,...,...,...
64537,9N3A7M2H07S,9,N3A7M2H07S,45,45,0
64538,9N3A7MD3EN,9,N3A7MD3EN,33,33,0
64539,9N3A7N0A00S,9,N3A7N0A00S,81,81,0
64540,9N3A7N1007S,9,N3A7N1007S,75,75,0


In [18]:
TD_STK = TD_STK.groupby('FILIAL')[['PRESENTE', 'STK_REAL', 'DIF_ABS']].sum().reset_index()
display(TD_STK)

,FILIAL,PRESENTE,STK_REAL,DIF_ABS
0,10,5075,5056,45
1,100,2904,2901,59
2,1000,931,938,13
3,1004,2097,2088,21
4,1005,5790,5776,16
...,...,...,...,...
524,993,5377,5362,55
525,994,5231,5424,215
526,996,6877,6875,24
527,997,1300,1297,7


In [19]:
TD_STK['Porcentagem%'] = (TD_STK['DIF_ABS'] / TD_STK['PRESENTE'])*100
TD_STK['Porcentagem%'] = TD_STK['Porcentagem%'].replace([np.inf, -np.inf], np.nan)
TD_STK['Porcentagem%'] = TD_STK['Porcentagem%'].fillna(0).round(0).astype(int)



In [20]:
TD_STK

,FILIAL,PRESENTE,STK_REAL,DIF_ABS,Porcentagem%
0,10,5075,5056,45,1
1,100,2904,2901,59,2
2,1000,931,938,13,1
3,1004,2097,2088,21,1
4,1005,5790,5776,16,0
...,...,...,...,...,...
524,993,5377,5362,55,1
525,994,5231,5424,215,4
526,996,6877,6875,24,0
527,997,1300,1297,7,1


In [21]:
condicoes = [
    TD_STK["DIF_ABS"] == 0,
    TD_STK["Porcentagem%"].between(0,2, inclusive="both"),
    TD_STK["Porcentagem%"].between(3,5, inclusive="both"),
    TD_STK["Porcentagem%"].between(6,9, inclusive="both"),
    TD_STK["Porcentagem%"] >= 10,


]

liberado_choices = ['OK','OK','OK', 'OK', 'NÂO']
obs_choices = ['Sem Diferença', 'Até 2%', 'Até 5%', 'Abaixo de 10%', 'Analisar']

TD_STK['Liberado'] = np.select(condicoes, liberado_choices, default='Revisar Regra')
TD_STK['Obs'] = np.select(condicoes, obs_choices, default='Revisar Regra')

TD_STK['Obs2'] = ''

TD_STK





,FILIAL,PRESENTE,STK_REAL,DIF_ABS,Porcentagem%,Liberado,Obs,Obs2
0,10,5075,5056,45,1,OK,Até 2%,
1,100,2904,2901,59,2,OK,Até 2%,
2,1000,931,938,13,1,OK,Até 2%,
3,1004,2097,2088,21,1,OK,Até 2%,
4,1005,5790,5776,16,0,OK,Até 2%,
...,...,...,...,...,...,...,...,...
524,993,5377,5362,55,1,OK,Até 2%,
525,994,5231,5424,215,4,OK,Até 5%,
526,996,6877,6875,24,0,OK,Até 2%,
527,997,1300,1297,7,1,OK,Até 2%,


In [22]:
print("Iniciando a etapa 2: Reclassificação dos grupos 'Analisar'")

filiais_para_analisar = TD_STK.loc[TD_STK['Obs'] == 'Analisar', 'FILIAL'].unique()
filiais_para_analisar

Iniciando a etapa 2: Reclassificação dos grupos 'Analisar'


array(['15'], dtype=object)

In [23]:
TD_STK_analisar =  df_check.groupby(['chave','FILIAL', 'ARTIGO_COR'])['PRESENTE'].sum().reset_index()
TD_STK_analisar = TD_STK_analisar.merge(df_estoque[['chave', 'STK_REAL']], how='left', on='chave' )
TD_STK_analisar['STK_REAL'] = TD_STK_analisar['STK_REAL'].fillna(0).astype(np.int64)
TD_STK_analisar['DIF_ABS'] = (TD_STK_analisar['PRESENTE'] - TD_STK_analisar['STK_REAL']).abs()
TD_STK_analisar = TD_STK_analisar[TD_STK_analisar['FILIAL'].isin(filiais_para_analisar)]
TD_STK_analisar

,chave,FILIAL,ARTIGO_COR,PRESENTE,STK_REAL,DIF_ABS
19730,150111AX7EN,15,0111AX7EN,21,3,18
19731,150111M2H07S,15,0111M2H07S,34,14,20
19732,150111MD307S,15,0111MD307S,32,-5,37
19733,150111N0A00S,15,0111N0A00S,26,-2,28
19734,150111N1007S,15,0111N1007S,38,14,24
...,...,...,...,...,...,...
19866,15N3A7M2H07S,15,N3A7M2H07S,45,-1,46
19867,15N3A7MD3EN,15,N3A7MD3EN,92,25,67
19868,15N3A7N0A00S,15,N3A7N0A00S,47,-10,57
19869,15N3A7N1007S,15,N3A7N1007S,100,19,81


In [24]:
def reavaliar_analisar(row):
    # Se já foi liberado na primeira etapa, mantém como está
    if row['Obs'] != 'Analisar':
        return row['Liberado'], row['Obs'], row['Obs2']

    filial_atual = row['FILIAL']
    
    # Busca os itens dessa filial no DataFrame detalhado
    # Ajuste 'TD_STK_analisar' se o nome do seu df detalhado for outro
    itens_filial = TD_STK_analisar[TD_STK_analisar['FILIAL'] == filial_atual]
    
    # Se não achar detalhes, retorna o que já estava
    if itens_filial.empty:
        return row['Liberado'], row['Obs'], "Sem detalhes encontrados"

    # Pega os 2 artigos com maior DIF_ABS
    top2_itens = itens_filial.sort_values(by='DIF_ABS', ascending=False).head(2)
    
    # Soma das diferenças dos 2 maiores
    soma_dif_top2 = top2_itens['DIF_ABS'].sum()
    artigos_ids = top2_itens['ARTIGO_COR'].tolist() # Para mensagem
    
    # Cálculo da NOVA Porcentagem (subtraindo os 2 maiores da diferença total)
    # Fórmula: (DIF_TOTAL - DIF_TOP2) / PRESENTE_TOTAL
    nova_dif = row['DIF_ABS'] - soma_dif_top2
    
    # Evita divisão por zero
    novo_perc = (nova_dif / row['PRESENTE'] * 100) if row['PRESENTE'] > 0 else 100
    novo_perc = round(novo_perc, 0) # Arredonda para facilitar comparação

    # --- LÓGICA DE DECISÃO ---
    
    # 1. Se baixou para 9% ou menos
    if novo_perc <= 9:
        return 'OK', 'Analisar', f"Liberar, exceto os artigos-cores {artigos_ids}"
    
    # 2. Se Presente > Stk Real E nova porcentagem entre 10% e 14%
    elif (row['PRESENTE'] > row['STK_REAL']) and (10 <= novo_perc <= 14):
        return 'OK', 'Analisar', f"Liberar, exceto os dois artigos cores, presente > stk Qlik {artigos_ids}"
    
    
    elif (row['PRESENTE'] < row['STK_REAL']):
        return 'NÃO', 'Analisar', "Muita diferença em vários artigos Presente < Stk Qlik"
    
    # 4. Caso residual (entre 10-14 mas Presente < Stk, ou entre 14-16, etc)
    else:
        return 'NÃO', 'Analisar', "Muita diferença em vários artigos"

# Aplica a função linha a linha (Apply)
# O zip agrupa os resultados para podermos atribuir a 3 colunas de uma vez
#Juntando as partes: TD_STK.apply(lambda row: ..., axis=1) significa:
#"Para cada linha (que chamaremos de row) no DataFrame TD_STK, execute o código que vem a seguir".
TD_STK[['Liberado', 'Obs', 'Obs2']] = TD_STK.apply(
    lambda row: pd.Series(reavaliar_analisar(row)), axis=1
)

# Exibe o resultado final
TD_STK

,FILIAL,PRESENTE,STK_REAL,DIF_ABS,Porcentagem%,Liberado,Obs,Obs2
0,10,5075,5056,45,1,OK,Até 2%,
1,100,2904,2901,59,2,OK,Até 2%,
2,1000,931,938,13,1,OK,Até 2%,
3,1004,2097,2088,21,1,OK,Até 2%,
4,1005,5790,5776,16,0,OK,Até 2%,
...,...,...,...,...,...,...,...,...
524,993,5377,5362,55,1,OK,Até 2%,
525,994,5231,5424,215,4,OK,Até 5%,
526,996,6877,6875,24,0,OK,Até 2%,
527,997,1300,1297,7,1,OK,Até 2%,


In [25]:
# Cria o agrupamento por 'Liberado' e 'Obs' e conta o tamanho (qtd de linhas)
resumo = TD_STK.groupby(['Liberado', 'Obs']).size().reset_index(name='Total')

# (Opcional) Ordena para ficar mais fácil de ler (Do maior para o menor)
resumo = resumo.sort_values(by='Total', ascending=True)

# Exibe o resumo
resumo



,Liberado,Obs,Total
0,NÃO,Analisar,1
4,OK,Sem Diferença,1
1,OK,Abaixo de 10%,4
3,OK,Até 5%,36
2,OK,Até 2%,487


In [26]:

total_filiais = resumo['Total'].sum()
print(f"Total de registros análise estoque {total_filiais}")

Total de registros análise estoque 529


In [27]:
# 1. Primeiro filtramos apenas as linhas onde Liberado é 'NÃO'
df_travado = TD_STK[TD_STK['Liberado'] == 'NÃO']

# 2. Agrupamos por Filial e Obs2, somando a coluna DIF_ABS
# O reset_index(name='Total') cria a coluna com o nome 'Total' igual ao seu print
resumo_travados = df_travado.groupby(['FILIAL', 'Obs2'])['DIF_ABS'].sum().reset_index(name='Total')

# (Opcional) Ordenar do maior para o menor erro total
resumo_travados = resumo_travados.sort_values(by='Total', ascending=False)

pd.set_option('display.max_colwidth', None)

# Agora exiba o resumo novamente
resumo_travados

,FILIAL,Obs2,Total
0,15,Muita diferença em vários artigos,4880


In [28]:
df_ressalvas = TD_STK[(TD_STK['Liberado'] == 'OK') & (TD_STK['Obs2'] != '')]

# 2. Agrupa por FILIAL e Obs2, somando o DIF_ABS
resumo_ressalvas = df_ressalvas.groupby(['FILIAL', 'Obs2'])['DIF_ABS'].sum().reset_index(name='Total')

# 3. Ordena do maior Total para o menor (opcional, ajuda a priorizar)
resumo_ressalvas = resumo_ressalvas.sort_values(by='Total', ascending=False)

# 4. Garante que o texto da Obs2 apareça inteiro
pd.set_option('display.max_colwidth', None)

# Exibe a tabela
resumo_ressalvas

,FILIAL,Obs2,Total


In [29]:
df_cat

,Artigo,PontoVda,Sit,Usuario,Pecas
0,SNG74PSN,1,ATENDIDO,SAP,2.0
1,SNG7WBDSN,1,ATENDIDO,SAP,1.0
2,SNG6WBDSN,1,ATENDIDO,SAP,3.0
3,SNG6AZ3SN,1,ATENDIDO,SAP,1.0
4,SNECN10SN,1,ATENDIDO,SAP,1.0
...,...,...,...,...,...
269182,0B5DHNJEN,-,SUSPENSO COMERCIAL + FINANCEIRO,WISE,29.0
269183,0B5DHNJEN,-,SUSPENSO FINANCEIRO,WISE,170.0
269184,0B5DHKMEN,-,ATENDIDO,SAP,11.0
269185,0B5DHKMEN,-,SUSPENSO COMERCIAL,SAP,16.0


In [30]:
df_cat.dtypes

Artigo        object
PontoVda      object
Sit           object
Usuario       object
Pecas        float64
dtype: object

In [31]:
df_cat.columns

Index(['Artigo ', 'PontoVda ', 'Sit ', 'Usuario ', 'Pecas '], dtype='object')

In [32]:
df_cat.columns = df_cat.columns.str.strip()

colunas_numericas = ['Pecas']
colunas_categoricas = ['Artigo', 'PontoVda', 'Sit', 'Usuario']

# --- AQUI ESTÁ A CORREÇÃO ---
# Usamos um loop caso você adicione mais colunas numéricas no futuro
for col in colunas_numericas:
    # Correção: to_numeric -> round(0) -> astype('Int64')
    df_cat[col] = pd.to_numeric(df_cat[col], errors='coerce').round(0).astype('Int64') 

# As categóricas continuam igual
df_cat[colunas_categoricas] = df_cat[colunas_categoricas].astype('str')

print("\n\n---- Depois da limpeza ---")
df_cat.dtypes



---- Depois da limpeza ---


Artigo      object
PontoVda    object
Sit         object
Usuario     object
Pecas        Int64
dtype: object

In [33]:
df_cat

,Artigo,PontoVda,Sit,Usuario,Pecas
0,SNG74PSN,1,ATENDIDO,SAP,2
1,SNG7WBDSN,1,ATENDIDO,SAP,1
2,SNG6WBDSN,1,ATENDIDO,SAP,3
3,SNG6AZ3SN,1,ATENDIDO,SAP,1
4,SNECN10SN,1,ATENDIDO,SAP,1
...,...,...,...,...,...
269182,0B5DHNJEN,-,SUSPENSO COMERCIAL + FINANCEIRO,WISE,29
269183,0B5DHNJEN,-,SUSPENSO FINANCEIRO,WISE,170
269184,0B5DHKMEN,-,ATENDIDO,SAP,11
269185,0B5DHKMEN,-,SUSPENSO COMERCIAL,SAP,16


In [34]:
df_cat['chave'] = df_cat['PontoVda'] + df_cat['Artigo']
df_cat

,Artigo,PontoVda,Sit,Usuario,Pecas,chave
0,SNG74PSN,1,ATENDIDO,SAP,2,1SNG74PSN
1,SNG7WBDSN,1,ATENDIDO,SAP,1,1SNG7WBDSN
2,SNG6WBDSN,1,ATENDIDO,SAP,3,1SNG6WBDSN
3,SNG6AZ3SN,1,ATENDIDO,SAP,1,1SNG6AZ3SN
4,SNECN10SN,1,ATENDIDO,SAP,1,1SNECN10SN
...,...,...,...,...,...,...
269182,0B5DHNJEN,-,SUSPENSO COMERCIAL + FINANCEIRO,WISE,29,-0B5DHNJEN
269183,0B5DHNJEN,-,SUSPENSO FINANCEIRO,WISE,170,-0B5DHNJEN
269184,0B5DHKMEN,-,ATENDIDO,SAP,11,-0B5DHKMEN
269185,0B5DHKMEN,-,SUSPENSO COMERCIAL,SAP,16,-0B5DHKMEN


In [35]:
df_cat["DtEmiNota"] = ''
# 1. Remove a coluna do final e guarda numa variável temporária
coluna_para_mover = df_cat.pop('DtEmiNota')

# 2. Insere ela na posição 1 (Logo após 'Artigo')
# A sintaxe é: .insert(posicao_numerica, 'nome_coluna', conteudo)
df_cat.insert(1, 'DtEmiNota', coluna_para_mover)

# Exibe o resultado
df_cat

,Artigo,DtEmiNota,PontoVda,Sit,Usuario,Pecas,chave
0,SNG74PSN,,1,ATENDIDO,SAP,2,1SNG74PSN
1,SNG7WBDSN,,1,ATENDIDO,SAP,1,1SNG7WBDSN
2,SNG6WBDSN,,1,ATENDIDO,SAP,3,1SNG6WBDSN
3,SNG6AZ3SN,,1,ATENDIDO,SAP,1,1SNG6AZ3SN
4,SNECN10SN,,1,ATENDIDO,SAP,1,1SNECN10SN
...,...,...,...,...,...,...,...
269182,0B5DHNJEN,,-,SUSPENSO COMERCIAL + FINANCEIRO,WISE,29,-0B5DHNJEN
269183,0B5DHNJEN,,-,SUSPENSO FINANCEIRO,WISE,170,-0B5DHNJEN
269184,0B5DHKMEN,,-,ATENDIDO,SAP,11,-0B5DHKMEN
269185,0B5DHKMEN,,-,SUSPENSO COMERCIAL,SAP,16,-0B5DHKMEN


In [36]:
df_cat_fat_hoje_ontem

,Artigo,DtEmiNota,PontoVda,Sit,Usuario,Pecas
0,SNKB1BSN,2025-12-02,-,FATURADO,SAP,3.0
1,SNKA1BSN,2025-12-02,-,FATURADO,SAP,1.0
2,SNK1N10SN,2025-12-02,-,FATURADO,SAP,2.0
3,SNJL7VSN,2025-12-02,-,FATURADO,SAP,1.0
4,SNJHNMCSN,2025-12-02,999,FATURADO,SAP,1.0
...,...,...,...,...,...,...
7296,77T61ASI,2025-12-03,-,FATURADO,SAP,1.0
7297,76WXEACEN,2025-12-03,-,FATURADO,SAP,1.0
7298,76WX12AEN,2025-12-03,-,FATURADO,SAP,1.0
7299,7C6ZHMJEN,2025-12-03,-,FATURADO,SAP,1.0


In [37]:
df_cat_fat_hoje_ontem.columns

Index(['Artigo ', 'DtEmiNota ', 'PontoVda ', 'Sit ', 'Usuario ', 'Pecas '], dtype='object')

In [38]:
df_cat_fat_hoje_ontem.columns = df_cat_fat_hoje_ontem.columns.str.strip()

colunas_numericas = ['Pecas']
colunas_categoricas = ['Artigo', 'DtEmiNota', 'PontoVda', 'Sit', 'Usuario']

# --- AQUI ESTÁ A CORREÇÃO ---
# Usamos um loop caso você adicione mais colunas numéricas no futuro
for col in colunas_numericas:
    # Correção: to_numeric -> round(0) -> astype('Int64')
    df_cat_fat_hoje_ontem[col] = pd.to_numeric(df_cat_fat_hoje_ontem[col], errors='coerce').round(0).astype('Int64') 

# As categóricas continuam igual
df_cat_fat_hoje_ontem[colunas_categoricas] = df_cat_fat_hoje_ontem[colunas_categoricas].astype('str')

print("\n\n---- Depois da limpeza ---")
df_cat_fat_hoje_ontem.dtypes



---- Depois da limpeza ---


Artigo       object
DtEmiNota    object
PontoVda     object
Sit          object
Usuario      object
Pecas         Int64
dtype: object

In [39]:
df_cat_fat_hoje_ontem['chave'] = df_cat_fat_hoje_ontem['PontoVda'] + df_cat_fat_hoje_ontem['Artigo']
df_cat_fat_hoje_ontem

,Artigo,DtEmiNota,PontoVda,Sit,Usuario,Pecas,chave
0,SNKB1BSN,2025-12-02,-,FATURADO,SAP,3,-SNKB1BSN
1,SNKA1BSN,2025-12-02,-,FATURADO,SAP,1,-SNKA1BSN
2,SNK1N10SN,2025-12-02,-,FATURADO,SAP,2,-SNK1N10SN
3,SNJL7VSN,2025-12-02,-,FATURADO,SAP,1,-SNJL7VSN
4,SNJHNMCSN,2025-12-02,999,FATURADO,SAP,1,999SNJHNMCSN
...,...,...,...,...,...,...,...
7296,77T61ASI,2025-12-03,-,FATURADO,SAP,1,-77T61ASI
7297,76WXEACEN,2025-12-03,-,FATURADO,SAP,1,-76WXEACEN
7298,76WX12AEN,2025-12-03,-,FATURADO,SAP,1,-76WX12AEN
7299,7C6ZHMJEN,2025-12-03,-,FATURADO,SAP,1,-7C6ZHMJEN


In [40]:
df_cat_fat_hoje_ontem["DtEmiNota"] = ''
# 1. Remove a coluna do final e guarda numa variável temporária
coluna_para_mover = df_cat_fat_hoje_ontem.pop('DtEmiNota')

# 2. Insere ela na posição 1 (Logo após 'Artigo')
# A sintaxe é: .insert(posicao_numerica, 'nome_coluna', conteudo)
df_cat_fat_hoje_ontem.insert(1, 'DtEmiNota', coluna_para_mover)

# Exibe o resultado
df_cat_fat_hoje_ontem

,Artigo,DtEmiNota,PontoVda,Sit,Usuario,Pecas,chave
0,SNKB1BSN,,-,FATURADO,SAP,3,-SNKB1BSN
1,SNKA1BSN,,-,FATURADO,SAP,1,-SNKA1BSN
2,SNK1N10SN,,-,FATURADO,SAP,2,-SNK1N10SN
3,SNJL7VSN,,-,FATURADO,SAP,1,-SNJL7VSN
4,SNJHNMCSN,,999,FATURADO,SAP,1,999SNJHNMCSN
...,...,...,...,...,...,...,...
7296,77T61ASI,,-,FATURADO,SAP,1,-77T61ASI
7297,76WXEACEN,,-,FATURADO,SAP,1,-76WXEACEN
7298,76WX12AEN,,-,FATURADO,SAP,1,-76WX12AEN
7299,7C6ZHMJEN,,-,FATURADO,SAP,1,-7C6ZHMJEN


In [41]:

df_consolidado = pd.concat([df_cat, df_cat_fat_hoje_ontem], ignore_index=True)

# Exibe o resultado final
df_consolidado

,Artigo,DtEmiNota,PontoVda,Sit,Usuario,Pecas,chave
0,SNG74PSN,,1,ATENDIDO,SAP,2,1SNG74PSN
1,SNG7WBDSN,,1,ATENDIDO,SAP,1,1SNG7WBDSN
2,SNG6WBDSN,,1,ATENDIDO,SAP,3,1SNG6WBDSN
3,SNG6AZ3SN,,1,ATENDIDO,SAP,1,1SNG6AZ3SN
4,SNECN10SN,,1,ATENDIDO,SAP,1,1SNECN10SN
...,...,...,...,...,...,...,...
276483,77T61ASI,,-,FATURADO,SAP,1,-77T61ASI
276484,76WXEACEN,,-,FATURADO,SAP,1,-76WXEACEN
276485,76WX12AEN,,-,FATURADO,SAP,1,-76WX12AEN
276486,7C6ZHMJEN,,-,FATURADO,SAP,1,-7C6ZHMJEN


In [42]:
df_consolidado['Pecas'].sum()

np.int64(6361733)

In [43]:
df_CA_agrupado = df_consolidado.groupby('chave')['Pecas'].sum().reset_index()
df_CA_agrupado
total = df_CA_agrupado['Pecas'].sum()

print("--- Estoque Agrupado por Chave ---")
print(df_CA_agrupado)
print(f"\n O total de estoque real no BI é: {total}")

--- Estoque Agrupado por Chave ---
             chave  Pecas
0       -0111A6PEN    634
1       -0111AX7EN    206
2       -0111AZ2EN    499
3       -0111H04EN    601
4       -0111HLQEN     10
...            ...    ...
241004  9N1TERA1EN     22
241005  9N3A7A0LEN      1
241006  9N3A7AX7EN      4
241007  9N3A7HMJEN      3
241008  9N3A7WHBEN      4

[241009 rows x 2 columns]

 O total de estoque real no BI é: 6361733


In [44]:
TD_CA =  df_check.groupby(['chave','FILIAL', 'ARTIGO_COR'])[['RESERVADO', 'EMBALADO', 'TRANSITO']].sum().reset_index()
TD_CA = TD_CA.merge(df_CA_agrupado[['chave', 'Pecas']], how='left', on='chave' )
TD_CA['CA_QLIK'] = TD_CA['Pecas'].fillna(0).astype(np.int64)
TD_CA['DIF_ABS'] = (TD_CA['RESERVADO'] - TD_CA['CA_QLIK']).abs()
TD_CA = TD_CA.drop(columns=['Pecas'])
TD_CA

,chave,FILIAL,ARTIGO_COR,RESERVADO,EMBALADO,TRANSITO,CA_QLIK,DIF_ABS
0,1000111AX7EN,100,0111AX7EN,0,0,0,0,0
1,1000111M2H07S,100,0111M2H07S,0,0,0,0,0
2,1000111MD307S,100,0111MD307S,0,0,0,0,0
3,1000111N0A00S,100,0111N0A00S,0,0,0,0,0
4,1000111N1007S,100,0111N1007S,0,0,0,0,0
...,...,...,...,...,...,...,...,...
64537,9N3A7M2H07S,9,N3A7M2H07S,0,0,0,0,0
64538,9N3A7MD3EN,9,N3A7MD3EN,0,0,0,0,0
64539,9N3A7N0A00S,9,N3A7N0A00S,0,0,20,0,0
64540,9N3A7N1007S,9,N3A7N1007S,0,0,0,0,0


In [45]:
df_relatorio

,Cod. Loja,Artigo,Cor,Cancelada,Atendido (SAP)
0,10,015M,MD6EN,0,1
1,10,015M,MD6EN,0,4
2,10,015M,N0A00S,0,4
3,10,015M,N0A00S,0,1
4,10,015M,N1007S,0,2
...,...,...,...,...,...
46672,282,N1TE,N1007S,0,2
46673,282,N1TE,N1007S,0,5
46674,282,N1TE,N1007S,0,4
46675,282,N1TE,N1007S,0,2


In [46]:
df_relatorio.columns

Index(['Cod. Loja', 'Artigo', 'Cor', 'Cancelada', 'Atendido (SAP)'], dtype='object')

In [47]:
df_relatorio.dtypes

Cod. Loja          int64
Artigo            object
Cor               object
Cancelada          int64
Atendido (SAP)     int64
dtype: object

In [48]:
df_relatorio['Cod. Loja'] = df_relatorio['Cod. Loja'].astype(str)

In [49]:
df_relatorio['chave'] = df_relatorio['Cod. Loja'] + df_relatorio['Artigo'] + df_relatorio['Cor']

In [50]:
df_relatorio = df_relatorio.groupby('chave')[['Atendido (SAP)', 'Cancelada']].sum().reset_index()
df_relatorio
total = df_relatorio['Atendido (SAP)'].sum()

print("--- Estoque Agrupado por Chave ---")
print(df_relatorio)
print(f"\n O total de estoque real no BI é: {total}")

--- Estoque Agrupado por Chave ---
               chave  Atendido (SAP)  Cancelada
0       1000201AX7EN               9          0
1       1000201HMJEN               1          0
2      1000201MD307S               3          0
3      1000201N0A00S              25          0
4      1000201N1007S               2          0
...              ...             ...        ...
19893      9KZGR1ASN               1          0
19894    9N1TEM2H07S               3          0
19895    9N1TEN0A00S               6          0
19896    9N1TEN1007S               2          0
19897    9N3A7N0A00S              20          0

[19898 rows x 3 columns]

 O total de estoque real no BI é: 75989


In [51]:
TD_CA = TD_CA.merge(df_relatorio[['chave', 'Atendido (SAP)', 'Cancelada']], how='left', on='chave' )
TD_CA['Atendido (SAP)'] = TD_CA['Atendido (SAP)'].fillna(0).astype(np.int64)
TD_CA['Cancelada'] = TD_CA['Cancelada'].fillna(0).astype(np.int64)
TD_CA

,chave,FILIAL,ARTIGO_COR,RESERVADO,EMBALADO,TRANSITO,CA_QLIK,DIF_ABS,Atendido (SAP),Cancelada
0,1000111AX7EN,100,0111AX7EN,0,0,0,0,0,0,0
1,1000111M2H07S,100,0111M2H07S,0,0,0,0,0,0,0
2,1000111MD307S,100,0111MD307S,0,0,0,0,0,0,0
3,1000111N0A00S,100,0111N0A00S,0,0,0,0,0,0,0
4,1000111N1007S,100,0111N1007S,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
64537,9N3A7M2H07S,9,N3A7M2H07S,0,0,0,0,0,0,0
64538,9N3A7MD3EN,9,N3A7MD3EN,0,0,0,0,0,0,0
64539,9N3A7N0A00S,9,N3A7N0A00S,0,0,20,0,0,20,0
64540,9N3A7N1007S,9,N3A7N1007S,0,0,0,0,0,0,0


In [52]:
TD_CA['RESERVADO'].sum()

np.int64(125874)

In [53]:
TD_CA['CA_QLIK'].sum()

np.int64(127037)

In [54]:
TD_CA['Atendido (SAP)'].sum()

np.int64(72073)

In [55]:
TD_CA['DIF_ABS'].sum()

np.int64(1799)

In [56]:
TD_CA['Cancelada'].sum()

np.int64(16477)

In [57]:
condicoes = [
    (TD_CA['RESERVADO'] > 0) & (TD_CA['CA_QLIK'] > 0),   # 1. Ambos maiores que 0
    (TD_CA['RESERVADO'] == 0) & (TD_CA['CA_QLIK'] > 0),  # 2. Reservado zerado, CA Qlik positivo
    (TD_CA['RESERVADO'] == 0) & (TD_CA['CA_QLIK'] == 0), # 3. Ambos zerados
    (TD_CA['RESERVADO'] > 0) & (TD_CA['CA_QLIK'] == 0)   # 4. Reservado positivo, CA Qlik zerado
]

# 2. Definimos os Cálculos Matemáticos para cada condição acima (na mesma ordem)
escolhas = [
    (TD_CA['RESERVADO'] - TD_CA['CA_QLIK']).abs(),       # 1. ABS(Reservado - Qlik)
    (TD_CA['CA_QLIK'] - TD_CA['TRANSITO']).abs(),        # 2. ABS(Qlik - Transito) -> Atenção aqui!
    TD_CA['RESERVADO'] - TD_CA['CA_QLIK'],               # 3. Reservado - Qlik
    TD_CA['RESERVADO'] - TD_CA['CA_QLIK']                # 4. Reservado - Qlik
]

# 3. Cria a coluna aplicando as regras
# O 'default=0' é caso alguma linha não caia em nenhuma regra (segurança)
TD_CA['dif_abs_trs'] = np.select(condicoes, escolhas, default=0)

# Exibe o resultado conferindo as colunas envolvidas
TD_CA[['RESERVADO', 'CA_QLIK', 'TRANSITO', 'dif_abs_trs']].head(10)

,RESERVADO,CA_QLIK,TRANSITO,dif_abs_trs
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
5,0,0,0,0
6,0,0,0,0
7,0,0,0,0
8,0,0,0,0
9,9,9,9,0


In [58]:
# A condição traduzida do Excel:
# E(J5<=K5; J5=F5)  --->  (Atendido <= Transito) & (Atendido == CA_QLIK)
condicao = (TD_CA['Atendido (SAP)'] <= TD_CA['TRANSITO']) & (TD_CA['Atendido (SAP)'] == TD_CA['CA_QLIK'])

# Aplica a lógica: np.where(condicao, valor_se_verdadeiro, valor_se_falso)
TD_CA['dif_aten_qlik'] = np.where(
    condicao, 
    (TD_CA['Atendido (SAP)'] - TD_CA['CA_QLIK']).abs(),  # Se verdadeiro
    TD_CA['DIF_ABS']                                     # Senão (pega o valor que já existe na DIF_ABS)
)

# Exibe o resultado para conferência
TD_CA[['Atendido (SAP)', 'TRANSITO', 'CA_QLIK', 'DIF_ABS', 'dif_aten_qlik']].head(10)

,Atendido (SAP),TRANSITO,CA_QLIK,DIF_ABS,dif_aten_qlik
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0
5,0,0,0,0,0
6,0,0,0,0,0
7,0,0,0,0,0
8,0,0,0,0,0
9,9,9,9,0,0


In [59]:
TD_CA

,chave,FILIAL,ARTIGO_COR,RESERVADO,EMBALADO,TRANSITO,CA_QLIK,DIF_ABS,Atendido (SAP),Cancelada,dif_abs_trs,dif_aten_qlik
0,1000111AX7EN,100,0111AX7EN,0,0,0,0,0,0,0,0,0
1,1000111M2H07S,100,0111M2H07S,0,0,0,0,0,0,0,0,0
2,1000111MD307S,100,0111MD307S,0,0,0,0,0,0,0,0,0
3,1000111N0A00S,100,0111N0A00S,0,0,0,0,0,0,0,0,0
4,1000111N1007S,100,0111N1007S,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
64537,9N3A7M2H07S,9,N3A7M2H07S,0,0,0,0,0,0,0,0,0
64538,9N3A7MD3EN,9,N3A7MD3EN,0,0,0,0,0,0,0,0,0
64539,9N3A7N0A00S,9,N3A7N0A00S,0,0,20,0,0,20,0,0,0
64540,9N3A7N1007S,9,N3A7N1007S,0,0,0,0,0,0,0,0,0


In [60]:
CA_agrupado = TD_CA.groupby('FILIAL')[['RESERVADO', 'CA_QLIK', 'EMBALADO', 'TRANSITO', 'DIF_ABS', 'Cancelada', 'dif_abs_trs', 'dif_aten_qlik']].sum().reset_index()

CA_agrupado

,FILIAL,RESERVADO,CA_QLIK,EMBALADO,TRANSITO,DIF_ABS,Cancelada,dif_abs_trs,dif_aten_qlik
0,10,375,375,0,926,0,0,0,0
1,100,240,271,226,640,31,0,22,31
2,1000,108,108,0,342,0,0,0,0
3,1004,106,106,93,69,0,0,0,0
4,1005,355,335,0,436,20,0,20,20
...,...,...,...,...,...,...,...,...,...
524,993,999,999,111,389,0,0,0,0
525,994,2,2,0,619,0,94,0,0
526,996,58,41,0,567,17,0,17,17
527,997,9,9,0,15,0,1186,0,0


In [61]:

condicoes_base = [
    CA_agrupado['DIF_ABS'] == 0,
    (CA_agrupado['DIF_ABS'] > 0) & (CA_agrupado['DIF_ABS'] < 11),
    (CA_agrupado['DIF_ABS'] >= 11) & (CA_agrupado['DIF_ABS'] <= 59)
]
escolhas_liberar = ['Ok', 'Ok', 'Ok']
escolhas_obs = ['Sem diferença', 'Até 10 peças de diferença', 'Pouca Diferença']

# Cria as colunas iniciais
CA_agrupado['Liberar'] = np.select(condicoes_base, escolhas_liberar, default='Analisar')
CA_agrupado['Obs'] = np.select(condicoes_base, escolhas_obs, default='Verificar Avançado')


print("Mapeando suspensões por filial...")

# 1. Filtra tudo que está suspenso no Qlik (df_consolidado)
df_suspensos = df_consolidado[
    df_consolidado['Sit'].str.startswith('SUSPENSO', na=False)
]

# 2. Cria um CONJUNTO DE PARCEIROS (Filial, Chave)
# Exemplo: O conjunto vai ter dados assim: {('657', '53C21NEN'), ('100', 'XPTO...')}
# Usamos .astype(str) para garantir que 657 (número) vire '657' (texto) e bata a comparação
chaves_suspensas_por_filial = set(
    zip(df_suspensos['PontoVda'].astype(str), df_suspensos['chave'])
)

# --- FUNÇÃO DE REVALIDAÇÃO ---'

def revalidar_carteira(row):
    # Se já foi liberado, passa reto
    if row['Liberar'] == 'Ok':
        return row['Liberar'], row['Obs']
    
    # Guarda a filial atual como texto para usar na comparação
    filial_atual_str = str(row['FILIAL'])
    
    # Pega os detalhes desta filial na tabela TD_CA
    detalhes = TD_CA[TD_CA['FILIAL'] == row['FILIAL']]
    
    if detalhes.empty:
        return 'NÃO', 'Sem detalhes na TD_CA'

    # --- REGRAS ANTERIORES (Mantidas) ---
    top2_itens = detalhes.sort_values(by='DIF_ABS', ascending=False).head(2)
    soma_dif_top2 = top2_itens['DIF_ABS'].sum()
    artigos_ids_top2 = top2_itens['ARTIGO_COR'].tolist()
    dif_restante_abs = row['DIF_ABS'] - soma_dif_top2
    
    if dif_restante_abs <= 59:
        return 'Ok', "Pouca Diferença"
    elif row['dif_abs_trs'] <= 59:
        return 'Ok', 'Pouca diferença (Regra Trânsito)'
    elif row['dif_aten_qlik'] <= 59:
        return 'Ok', 'Pouca diferença (Regra Atendido)'

    top_item_aten = detalhes.sort_values(by='dif_aten_qlik', ascending=False).head(2)
    soma_item_top2 = top_item_aten['dif_aten_qlik'].sum()
    artigos_ids_aten = top_item_aten['ARTIGO_COR'].tolist()
    saldo_restante_aten = row['dif_aten_qlik'] - soma_item_top2

    if saldo_restante_aten <= 59:
        return 'Ok', f"Liberar, exceto {artigos_ids_aten}"

    # --- SUA REGRA FINAL AJUSTADA (Cancelados + Suspensão Confirmada) ---
    else:
        # 1. Soma total de cancelados dessa filial
        total_cancelado = detalhes['Cancelada'].sum()
        
        # 2. Identifica quais chaves têm saldo no Qlik (CA_QLIK > 0)
        # (São esses itens que "sobram" e precisam estar justificados como suspensos)
        itens_com_saldo_qlik = detalhes[detalhes['CA_QLIK'] > 0]['chave'].tolist()
        
        # 3. VERIFICAÇÃO RIGOROSA:
        # "Para CADA item que tem saldo no Qlik, ele existe na lista de suspensos DESSA FILIAL?"
        todos_itens_estao_suspensos = True
        
        if len(itens_com_saldo_qlik) == 0:
            # Se não tem nenhum item com saldo Qlik, teoricamente não precisa checar suspensão
            todos_itens_estao_suspensos = False 
        else:
            for chave in itens_com_saldo_qlik:
                # O Pulo do Gato: Verifica se o par (Filial, Chave) existe no conjunto
                if (filial_atual_str, chave) not in chaves_suspensas_por_filial:
                    todos_itens_estao_suspensos = False
                    break # Se um falhar, já era, não é "Todos"
        
        # 4. APLICAÇÃO DA DECISÃO
        # Se cobriu o valor com cancelados E tudo que sobrou no Qlik está realmente suspenso nessa loja
        if (total_cancelado >= row['DIF_ABS']) and todos_itens_estao_suspensos:
            return 'NÃO', 'Todos os pedidos foram cancelados LW'
            
        else:
            return 'NÃO', 'Diferença Crítica'

    # --- EXECUÇÃO ---
print("Rodando validação com verificação de Filial+Chave...")

CA_agrupado[['Liberar', 'Obs']] = CA_agrupado.apply(
    lambda row: pd.Series(revalidar_carteira(row)), axis=1
)

# Exibe o resultado
CA_agrupado.head()

Mapeando suspensões por filial...
Rodando validação com verificação de Filial+Chave...


,FILIAL,RESERVADO,CA_QLIK,EMBALADO,TRANSITO,DIF_ABS,Cancelada,dif_abs_trs,dif_aten_qlik,Liberar,Obs
0,10,375,375,0,926,0,0,0,0,Ok,Sem diferença
1,100,240,271,226,640,31,0,22,31,Ok,Pouca Diferença
2,1000,108,108,0,342,0,0,0,0,Ok,Sem diferença
3,1004,106,106,93,69,0,0,0,0,Ok,Sem diferença
4,1005,355,335,0,436,20,0,20,20,Ok,Pouca Diferença


In [62]:

# Tente assim se o anterior não funcionar
CARLOS = ['100', '331', '346', '1284']
CA_agrupado_filtrado = CA_agrupado[CA_agrupado['FILIAL'].isin(CARLOS)]

# Agora exibe
display(CA_agrupado_filtrado)

,FILIAL,RESERVADO,CA_QLIK,EMBALADO,TRANSITO,DIF_ABS,Cancelada,dif_abs_trs,dif_aten_qlik,Liberar,Obs
1,100,240,271,226,640,31,0,22,31,Ok,Pouca Diferença
150,1284,406,437,281,9,31,0,31,31,Ok,Pouca Diferença
244,331,526,553,174,75,27,0,27,27,Ok,Pouca Diferença
251,346,195,195,185,385,0,0,0,0,Ok,Sem diferença


In [63]:
# Cria o agrupamento por 'Liberado' e 'Obs' e conta o tamanho (qtd de linhas)
resumo = CA_agrupado.groupby(['Liberar', 'Obs']).size().reset_index(name='Total')

# (Opcional) Ordena para ficar mais fácil de ler (Do maior para o menor)
resumo = resumo.sort_values(by='Total', ascending=False)

# O til (~) inverte a lógica: traz tudo que NÃO contém "exceto"
resumo_limpo = resumo[~resumo['Obs'].str.contains("exceto", case=False, na=False)]

print(resumo_limpo['Total'].sum())
# Exibe o resultado
resumo_limpo



529


,Liberar,Obs,Total
2,Ok,Sem diferença,388
0,Ok,Até 10 peças de diferença,76
1,Ok,Pouca Diferença,65


In [64]:
# 2. Agrupamos e SOMAMOS as colunas numéricas
# ADICIONE O .reset_index() NO FINAL AQUI 👇
CA_OK = CA_agrupado.groupby(['FILIAL', 'Obs'])[['DIF_ABS', 'dif_abs_trs', 'dif_aten_qlik']].sum().reset_index()

# Agora sim essa linha vai funcionar, pois 'Obs' voltou a ser uma coluna
resumo_OK = CA_OK[CA_OK['Obs'].str.contains("exceto", case=False, na=False)]

pd.set_option('display.max_rows', None)
resumo_OK

,FILIAL,Obs,DIF_ABS,dif_abs_trs,dif_aten_qlik


In [65]:
# 1. Primeiro filtramos apenas as linhas onde Liberado é 'NÃO'
CA_travado = CA_agrupado[CA_agrupado['Liberar'] == 'NÃO']

# 2. Agrupamos por Filial e Obs2, somando a coluna DIF_ABS
# O reset_index(name='Total') cria a coluna com o nome 'Total' igual ao seu print
CA_travado = CA_travado.groupby(['FILIAL', 'Obs'])['DIF_ABS'].sum().reset_index(name='Total')

# (Opcional) Ordenar do maior para o menor erro total
resumo_travados = CA_travado.sort_values(by='Total', ascending=False)

pd.set_option('display.max_colwidth', None)

# Agora exiba o resumo novamente
resumo_travados

,FILIAL,Obs,Total
